In [222]:
import os
import mido
from mido import MidiFile, MidiTrack, Message
import random
from numpy import random as nprnd

In [223]:
def generate_random(snote=50, mlength=12, numofmidi=10, pitchrnd=False, timernd=False):
    
    notes = range(snote, snote+mlength)
    times = range(50,300)
    time = 100 

    # pitch range for random pitch value ;
    pitches = range(-8192,8191)
    
    # Create music folder if it does not exist
    if not os.path.exists('music'):
        os.makedirs('music')

    for j in range(numofmidi):
    
        mid = MidiFile(type=0) # type0 can have only one track
    
        track = MidiTrack() # note list (kind of)

        mid.tracks.append(track)
    
        # the note which the pitch will change for
        pcnote = random.choice(noterange)
    
        for i in range(mlength):
        
            note = random.choice(notes)
            pitch = random.choice(pitches)
            
            if timernd:
                time = random.choice(times)
        
            if pitchrnd:
                # Change the pitch for only one note
                if i == pcnote: # Change the pitch on third note
                    track.append(Message('pitchwheel', pitch=pitch))
                if i == (pcnote+1): # Change the pitch back to default
                    track.append(Message('pitchwheel'))
        
            track.append(Message('note_on', channel=0, note=note, time=time))
            track.append(Message('note_off', channel=0, note=note, time=time))
        

        mid.save('music/random' + str(j) + '.mid')

In [224]:
def fitnessfunction(m,p):
    fitness = 1
    for x in range (1,mlength):
        fitness= fitness * p[m[x-1]][m[x]]
    return fitness

In [225]:
def mutate(notelist, mlength=12):
    
    prob = []    
    for x in range(0,12):
        a=nprnd.sample(12)
        a=a/sum(a)
        prob.append(a)
    
    bestscore = 0  
    progress = []
    mutaterange=[0,1,2,3,4,5,6,7,8,9,10,11]
    
    # notelist of the midi
    bestmelody = notelist
    
    #each loop decreases the number of mutations per round
    for i in range (0,5):

        #each loop mutates the currentbest, checks it's fitness and updates currentbest   
        for x in range (0,1000):
            melody=list(bestmelody)      
            r = nprnd.randint(0,mlength,abs(i-6))
            for z in r:
                mutaterangecopy=list(mutaterange)
                mutaterangecopy.remove(melody[z])
                melody[z]= nprnd.choice(mutaterangecopy,1)[0]
        
        
            currentscore = fitnessfunction(melody,prob)
            if currentscore >= bestscore:
                bestscore = currentscore
                bestmelody = list(melody)
        
            progress.append(bestscore)
            
    return bestmelody

In [226]:
def apply_mutation(mutantnotelist, midino, snote=50):

    mid = MidiFile(type=0) # type0 can have only one track
    
    track = MidiTrack() # note list (kind of)

    mid.tracks.append(track)
    
    time = 100
    
    # Create mutant music folder if it does not exist
    if not os.path.exists('mutantmusic'):
        os.makedirs('mutantmusic')
    
    # add the octaves back
    mutantnotelist2 = [x+snote for x in mutantnotelist]
    
    for note in mutantnotelist2:
        
        #print(note)
        
        track.append(Message('note_on', channel=0, note=int(note), time=time))
        track.append(Message('note_off', channel=0, note=int(note), time=time))
        
        
    mid.save('mutantmusic/random' + str(midino) + '.mid')

In [227]:
def read_midi(filename='music/random4.mid', snote=50):
    
    mid = MidiFile(filename)
    
    noteonlist = []
    for i, track in enumerate(mid.tracks):
        #print('Track {}: {}'.format(i, track.name))
        for message in track:
            #print(message)
            if message.type == 'note_on':
                noteonlist.append(message.note)
    
    # normalize the note integers for mutation by reducing octaves
    notelist = [x-snote for x in noteonlist]
    
    return notelist

In [228]:
mlength = 12
snote = 50 # starting note
numofmidi = 10 # number of midi files
pitch=False # do not include pitch variations for now

In [229]:
generate_random(snote, mlength, numofmidi, pitch)

for j in range(numofmidi):
    
    midiname = 'music/random' + str(j) + '.mid'
    
    notelist = read_midi(midiname)
    
    mutantnotelist = mutate(notelist)
    
    apply_mutation(mutantnotelist, j)
    
    